In [3]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

train = pd.read_csv("../data/train.csv", index_col=0)
test = pd.read_csv("../data/test.csv", index_col=0)

data = pd.concat([train, test], axis=0)
data.index = pd.to_datetime(data.index)

data['year'] = data.index.year
data['month'] = data.index.month
data['day'] = data.index.day
data['weekday'] = data.index.weekday
data['dayofweek'] = data.index.dayofweek #weekday를 대체 (weekday, dayofweek 어떤 피쳐든 점수 동일했음)
data['hour'] = data.index.hour
data['year_season'] = data['year'] + data['season'] / 10
    #data['year_season']: 년도와 계절 정보를 결합하여 새로운 특징을 생성
    #계절 정보가 소수점으로 추가되어, 각 년도의 계절을 구별할 수 있게 함

test_start = '2012-12-01'
train_data = data[data.index < test_start].copy()
test_data = data[data.index >= test_start].copy()


# 대여 수 ('cnt')에 로그 변환을 적용
train_data['cnt_log'] = np.log(train_data['cnt'] + 1)


#groupby와 median 함수를 사용하여 각 'year_season' 그룹별로 'cnt'의 중앙값을 계산합니다.
#이 중앙값을 'count_season'이라는 새로운 열로 데이터프레임에 추가하고, 이는 계절적 요인을 반영한 추가적인 특징이 됨.
by_season = train_data.groupby('year_season')['cnt'].median()
by_season = by_season.rename('count_season')


train_data = train_data.join(by_season, on='year_season')
test_data = test_data.join(by_season, on='year_season')

#두 가지 머신러닝 모델을 사용: 그래디언트 부스팅 (GBDT)과 랜덤 포레스트 (RF).
#random_state=0은 모델의 결과가 일관되게 재현될 수 있도록 설정
regs = {"gbdt": GradientBoostingRegressor(random_state=0),
        "rf": RandomForestRegressor(random_state=0, n_jobs=-1)}

features = ['season', 'holiday', 'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed', 'year', 'hour', 'dayofweek', 'count_season']

preds = {}
#각 모델에 대해 학습을 수행하고, 테스트 데이터에 대한 예측을 수행
for name, reg in regs.items():
    if name == 'gbdt':
        reg.set_params(n_estimators=1000, min_samples_leaf=6)
    elif name == 'rf':
        reg.set_params(n_estimators=1000, min_samples_leaf=2)

    reg.fit(train_data[features], train_data['cnt_log'])
    #이 단계에서는 특징(feature)과 대상 변수(target variable)를 모델에 제공
    
    preds[name] = reg.predict(test_data[features])
    preds[name] = np.exp(preds[name]) - 1 # 원본 scale로 transform

test_pred = 0.5 * preds['gbdt'] + 0.5 * preds['rf']
test_pred[test_pred < 0] = 0 #예측값이 음수인 경우를 0으로 처리. 자전거 대여 수는 음수가 될 수 없기 때문

In [4]:
submission_df = pd.read_csv('../data/sample_submission.csv')
submission_df['cnt'] = test_pred
submission_df.to_csv('../submission/final_submission1.csv', index=False)